# Library Carpentry: Tools for Humanists

## Python Lesson

### Part 2: Working with data using Pandas

This lesson is adapted from Melanie Walsh's [Introduction to Cultural Analytics with Python](https://melaniewalsh.github.io/Intro-Cultural-Analytics/03-Data-Analysis/01-Pandas-Basics-Part1.html) and Dolsy Smith's pandas lesson taught in [Python Camp](https://libguides.gwu.edu/python-camp). 

We're going to learn some ways to work with tabular data (such as in a CSV file) to do exploration, analysis, and plotting with Python. We'll be using a particular Python library called pandas. The name pandas comes from "Python Data Analysis" library. 

We will cover how to:

* Import Pandas
* Read in a CSV file
* Explore and filter data
* Make simple plots and data visualizations


The pandas library is already installed in the Google Colab environment, so we don't need to do anything special to install the module. We do, however, need to use the import command so that the Python interpreter will understand what we are referencing it in our code. 

We're going to import it and use a nickname, or shorter-to-type name for it. 

In [ ]:
 import pandas as pd

Pandas is very handy at reading in tabular data from CSV or Excel and turning it into a Python data structure we can use for analysis. 

First, we need to load the CSV into this Google Colab environment. 

We will use the function `pd.read_csv()` to load our CSV file from an external site, providing the function with the URL. 

This creates a Pandas DataFrame object — often you'll see people use df as a variable name to represent the DataFrame, but you can name it anything. 

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/gwu-libraries/2022-07-14-gwu/gh-pages/files/Cleaned-columbian-college-volumes.csv")
df

We can see that this is a new Python data type, a DataFrame, which we got when we imported the pandas library. 

A DataFrame looks and acts a lot like a spreadsheet. 

Not all the rows are shown, but we can see at the bottom that there are 459. We also have column labels across the top, and index labels down the left-hand side. Here the index labels correspond to the row numbers (as in a spreadsheet, with the notable difference of starting with 0) but that doesn’t have to be the case. 

Note that those numbers weren't part of our spreadsheet that we imported.

A DataFrame, like almost everything in Python, has a data type. But unlike a list or dictionary, a DataFrame is a user-defined type, meaning that the creators of the pandas library gave it special methods and properties we can use when working with it. 

In [ ]:
type(df)

We can also take a look at some of the data using a method belonging to the DataFrame object called head()

In [ ]:
df.head()


The head method by default only shows us the first five rows, although we can change that by providing the number of rows as an argument.

In [ ]:
df.head(20)

You're probably wondering what all of those NaN values are in many of the columns. That's a missing value in the data, and pandas uses the Python "Not a Number" value (it's technically actually a float) to act as a placeholder in the data. It's a null value. 

Determining how to deal with missing values is an important step in working with any data. Let's first look at how many missing values are in this data.

In [ ]:
df.info()

You can see that while there are 459 rows of data, some of the columns have fewer than 459 values. The remaining values are the NaN null values. We can see that the Series, Relation, and Medium Type columns are nearly or completely empty in this dataset. We'll get rid of those columns to make our data easier to work with (and smaller, which could be helpful for much larger datasets). 

Note also the Dtype column. That shows us the "type" of the column, much like a column in an Excel sheet has a type. In this case, "object" refers to strings or a column that has a mix of strings and numeric types. This is common with text data like our dataset. However you might also encounter datetime values or booleans or floats. 

See “[Working with missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html?highlight=nan/)” for more information in the pandas docs.

Here's another way to look at missing data: we can count the number of null values (instead of non-null). 

In [ ]:
#df.isnull() combining isnull() (returns True/False) with sum() gives us the number of null instead of the number of values
df.isnull().sum()

Let's remove those columns that aren't useful to us because they're practically all null values. 

In [ ]:
df = df.drop(columns=["Series", "Relation", "Medium Type"])
df

In [ ]:
df.info()

**Optional:**

There are ways to replace NaN values with a different value, such as 0 or a string "Unknown". `.fillna()`

`fillna()` operates on a column (or the entire dataframe) and takes as its first argument the value you want to have replace NaN. Then since we want to overwrite NaN in the DataFrame--change it in a way that persists rather than just outputting it, we add the keyword argument `inplace=True`. 

In [ ]:
df["Edition"].fillna("Unknown", inplace=True)
df

**Subsets of data** 

There are some ways to look at subsets of our data either by position in the DataFrame (like certain row numbers) or by criteria. 

We're going to use notation that looks a lot like slicing, in combination with keyword `loc`. It's not really a method. Loc lets us use the numeric position of a row. 

There's a notable difference between how loc works and how slicing works. Loc is *inclusive*, meaning that the last position is included in the slice. That is not how slicing works (as you saw on strings and lists). 

(Note we are using the row positions, which happen to be same as the index here, but it's not always!)

In [ ]:
df.loc[5:11]

We can select columns using their name. 

In [ ]:
# How to select columns
subjects = df["Subject"]
subjects

This returns a pandas Series. There is a numeric index for the values in the Series. 

We can combine column selection and using loc to get a particular or set of rows:

In [ ]:
# see full value of first element in Series. 
df["Subject"].loc[0]

How would you select only the Imprint column?

In [ ]:
df["Imprint"]

To more practical purposes, let's move on to selecting subsets of data that meet particular conditions. This is something we might actually want to do in working with a dataset, either as you're exploring it, or to do clean-up or analysis operations on some subset of the rows. You might already do something like this in Excel or Google Sheets with filters. 

To answer the question "How many books by Isaac Newton are in this collection?"  we need to create a test that returns True if the Creator is the value, "Newton, Isaac". 

First, let's just look at the column and test a condition. 

In [ ]:
df["Creator"] == "Newton, Isaac"

This returns a series of Trues and Falses. So not very valuable by itself. However, we can use this as a "mask" or filter on the entire DataFrame:

In [ ]:
df[df["Creator"] == "Newton, Isaac"]

## Exercise 2.1

1. Create a subset of volumes where language is Latin.

2. Create a subset of volumes where the language is Latin AND the author is Isaac Newton. 
Hints: 
* Put each condition in parentheses. 
* In pandas, you use the `|`and `&` characters to represent OR and AND. 


In [ ]:
# Answer to exercise 1:
df[df["Language"] == "Latin"]
df

In [ ]:
# Answer to exercise 2:
df[(df["Language"] == "Latin") & (df["Creator"] == "Newton, Isaac")]

Text datasets may not have a lot of repeated exactly equal values so you may find it more useful to look at strings that start with or contain particular substrings. To test whether a string contains another substring, we can use a different sort of syntax than the equality test we used before. In pandas, you can use the `.str.contains()` method. to test whether a string contains a pattern or regular expression. 

(This is where pandas syntax starts to feel particularly weird.)

In [ ]:
df[df["Imprint"].str.contains("London")]


If you looked closely at the dataset, however, you might notice that some of the Imprint statements have publication locations that are not in English. For example, London also shows up as Londres and Londini. We can use the `|` within the pattern string to OR together several possible substring patterns. 

In [ ]:
 df[df["Imprint"].str.contains("London|Londres|Londini")]

Our dataset is up to 181 rows now. 

If we wanted to create a new dataframe with just hte results of this filtering, we can assigned the result of the expression to a new variable, for example, `df2`:

In [ ]:
df2 = df[df["Imprint"].str.contains("London|Londres|Londini")]
df2

## Categorical data and plots

One reason to explore your data with pandas is that you can take the results of your filtering and clean-up and make plots for quick analysis. 

If we wanted to look at the distribution of different languages in the dataset, we can use pandas to get the number of times each language appears and then plot that as a bar chart. Language is a categorical variable, which is why a bar chart is useful. It counts the frequency of distinct categories. 

In [ ]:
#Categorical data: can counts frequency of values.

lang_counts = df["Language"].value_counts()
lang_counts

Note that sort_values returns the values in order, with the highest value first. 

Something to note about lang_counts is that it is a Series (not a DataFrame), and the index is the language name *not* a numeric index like we got when we were looking at columns earlier. Instead, it's the value being counted. We can access a particular language's count by index:

In [ ]:
type(lang_counts)

In [ ]:
lang_counts["Spanish"]

In [ ]:
lang_counts.loc["Spanish"]

## Make and Save Plots

Pandas makes it easy to create plots and data visualizations. We can make a simple plot by adding .plot() to any DataFrame or Series object that has appropriate numeric data. 

(In the case of our language data, the numeric data is the counts of each language value.)

We specify the title with the title= parameter and the kind of plot by altering the kind= parameter:


In [ ]:
lang_counts.plot(kind="bar", title="Number of Volumes by Language");

The number of volumes per language drops off quickly, so let's adjust the plot to only show the top ten most frequent languages. 


In [ ]:
lang_counts[:10].plot(kind="bar", title="Number of Volumes by Language");

Some more options to try in order to improve the formatting. It's easier to see what is available if we use a slightly different, more specific syntax with the bar() method.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.bar.html?#pandas.DataFrame.plot.bar

and for kwargs:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html#pandas.DataFrame.plot

In [ ]:
# color: str, array-like, or dict, optional. The color for each of the DataFrame’s columns.
# figsize: a tuple (width, height) in inches
# rot = rotation of ticks. Note that it messes up the long tickname.

lang_counts[:10].plot.bar(title="Number of Volumes by Language", 
                          color=["green", "blue", "red"],  
                          figsize=(10, 5), rot=45, grid=True);

#### Exercise 2.2 #####

1. Determine who the most frequent authors (or "Creators") are in the set. 

2. Use the same approach as what we used with language and make a chart showing the top 25 authors. Experiment with the formatting.

In [ ]:
author_counts = df["Creator"].value_counts()
author_counts

In [ ]:
# remove the first one that's Columbian College by changing slice
# Add a title and figsize
author_counts.iloc[1:25].plot.bar(figsize=(10,5), 
                                  title="Volumes by Author\n", 
                                  color="green", rot="80", fontsize=12);

## Regular expressions in pandas

There are many ways to apply regular expressions in pandas. We'll look at one approach, but this is not only way.  

Let's say we wanted to get the year from the "Imprint" column, We can use the pandas `str.extract()` method to use a regular expression on a column. Earlier, we used `str.contains` to test whether a value contained a particular substring. 

`str.extract()` will allow us to use a pattern in the form or a regex to match and then grab that matching substring. 

Here's one of the regualr expressions we came up with earlier to access a four-digit year#

`(.+)\s?[:,]\s(.+)[,;]\s(\d{4})`

We can apply that pattern to a whole column. 

https://pandas.pydata.org/docs/reference/api/pandas.Series.str.extract.html?#pandas.Series.str.extract

In [ ]:
df["Imprint"].str.extract('(.+)\s?[:,]\s(.+)[,;]\s(\d{4})')

# Note that not all Imprints will match and the result in those cases is NaN 

So why are there three columns in the returned DataFrame? The pattern we used has three capture groups, since we were trying to grab different parts initially. 

So, we could access just that last column, labeled "2" and make that our Year column, OR we could edit the statement to only have one capture group (here in cell below). 

`.+\s?[:,]\s.+[,;]\s(\d{4})`

`df["Imprint"].str.extract(".+\s?[:,]\s.+[,;]\s(\d{4})")`


In [ ]:
df["Year"] = df["Imprint"].str.extract('(.+)\s?[:,]\s(.+)[,;]\s(\d{4})')[2]
df

**END HERE.** 

Next up: more using these skills with another dataset for text analysis. 

**Extra content**
What's below is extra code related to alternative ways to work with regular expressions and using `.apply()`

In [ ]:
# Use regular expressions to extract the date from the first column and make a new column with that data. 

# Option 1 (not preferred?) is to use re.compile and then a function to apply it. 
import re
title = "Philadelphia, Bradford and Inskeep, 1808-14."
pattern = re.compile("(.+)\s?[:,]\s(.+)[,;]\s(\d{4})")

result = re.search(pattern, title)
print(result)
print(result[3])

In [ ]:
# alternatively use re.compile on the column. May be too much for this lesson. 

def get_year_without_try(title):
    pattern = re.compile("(.+)\s?[:,]\s(.+)[,;]\s(\d{4})")
    result = re.search(pattern, title)
    return result[3]


def get_year(title):
    pattern = re.compile("(.+)\s?[:,]\s(.+)[,;]\s(\d{4})")
    result = re.search(pattern, title)
    try:
        return result[3]
    except:
      return None

In [ ]:
# This will raise an error because of how None can not be accessed with an index

df["Year_Re"] = df["Imprint"].apply(get_year_without_try)

In [ ]:
# note last value is None

df["Year_Re"]

In [ ]:
df["Year_Re"] = df["Imprint"].apply(get_year)
df

From Python docs: Match objects always have a boolean value of True. Since match() and search() return None when there is no match, you can test whether there was a match. 

If search() finds multiple matches, only the last one is accessible. 

https://docs.python.org/3/library/re.html#match-objects